In [1]:
from myUtils import pickle_load
verbose = False  # True prints more output

In [2]:
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
filename_pickled_df_symbols_close = "df_symbols_close"  # pickled filename

In [ ]:
df_OHLCV = pickle_load(path_data_dump, 'df_OHLCV', verbose=verbose)

In [58]:
df_OHLCV['A'].to_csv('C:/Users/ping/Desktop/A_OHLCV.csv')

In [3]:
# load df_symbols_close
df = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_orig = df.copy()  # backup df
df = df.tail(252)  # keep only the last 252 days of data
symbols_before_dropna = df.columns

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close


In [4]:
df['ZS'].to_csv('C:/Users/ping/Desktop/ZS.csv')

In [5]:
df = df.dropna(axis=1,how='any')  # drop symbols (i.e. columns) with NaN
symbols_after_dropna = df.columns
symbols_dropped = symbols_before_dropna.difference(symbols_after_dropna).tolist()
# print(f'symbols with NaN dropped from dataframe :{symbols_dropped}')
print(f'{len(symbols_dropped)} symbols have NaN in their close price, and were dropped from the dataframe.')
print(f'These symbols were dropped:\n{symbols_dropped}')

44 symbols have NaN in their close price, and were dropped from the dataframe.
These symbols were dropped:
['ACT', 'BLCO', 'BRCC', 'BRZE', 'BXSL', 'CEG', 'CMTG', 'CRDO', 'CRGY', 'CWAN', 'DH', 'EE', 'ESAB', 'ESMT', 'FLNC', 'FRSH', 'GBTG', 'GFS', 'GTLB', 'HCP', 'HKD', 'HLN', 'IHS', 'INFA', 'IOT', 'JXN', 'KD', 'LTH', 'NU', 'OLPX', 'ONON', 'PFHC', 'RIVN', 'RTLR', 'SHEL', 'SLVM', 'SMR', 'SRAD', 'STER', 'TCN', 'TOST', 'TWKS', 'UDMY', 'WE']


In [9]:
df.shape

(252, 1873)

In [39]:
df.columns

Index(['A', 'AA', 'AAL', 'AAON', 'AAP', 'AAPL', 'AAWW', 'ABB', 'ABBV', 'ABC',
       ...
       'ZI', 'ZIM', 'ZION', 'ZLAB', 'ZM', 'ZNH', 'ZS', 'ZTO', 'ZTS', 'ZWS'],
      dtype='object', length=1873)

In [44]:
df['A'].to_csv('C:/Users/ping/Desktop/A.csv')
df['ZWS'].to_csv('C:/Users/ping/Desktop/ZWS.csv')

In [7]:
from myUtils import symb_perf_stats_vectorized_v1
symbols, period_yr, drawdown, UI, max_drawdown, \
        returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = symb_perf_stats_vectorized_v1(df)

In [10]:
print(f'drawdown.shape: {drawdown.shape}')
print(f'type(drawdown): {type(drawdown)}')

drawdown.shape: (252, 1873)
type(drawdown): <class 'numpy.ndarray'>


In [12]:
def moving_window(arr, window):
    # reference: http://www.rigtorp.se/2011/01/01/rolling-statistics-numpy.html
    """Create moving-window of numpy.ndarray from a numpy.ndarray or pandas.Series

    Args:
        arr(float): a numpy.ndarray or pandas.Series
        window(int): width of moving-window

    Return:
        numpy.ndarray with shape [len(arr) - window + 1, window]
            where each row has elements of the moving_window,
            and number of elements in each row is the window size (i.e. columns)
    """

    import numpy as np

    shape = arr.shape[:-1] + (arr.shape[-1] - window + 1, window)
    strides = arr.strides + (arr.strides[-1],)

    return np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)

In [13]:
def UI_MW(arr_drawdown, window=20):
    """Calculate Ulcer-Index on moving-window applied to
       arr_drawdown(numpy.ndarray)

    Args:
        arr_drawdown(numpy array): drawdown from function symb_perf_stats or
                                   from ulcer_index.py
        window(int): moving-window size, default = 20

    Return:
        UI_MW(numpy.ndarray): Ulcer-Index on moving-window
        applied to arr_drawdown
    """

    import numpy as np

    # apply moving window to arr_drawdown, result is numpy ndarray
    DD_MW = moving_window(arr_drawdown, window)
    DD_MW_square = np.square(DD_MW)
    sum_DD_MW_square = np.sum(DD_MW_square, axis=1)  # sum each row
    UI_MW = np.sqrt(sum_DD_MW_square / window)
    UI_MW = nan_pad(len(arr_drawdown), UI_MW)  # pad with leading NaN

    return UI_MW

In [47]:
def UI_MW_15(arr_drawdown):
    """Calculate Ulcer-Index on moving-window applied to
       arr_drawdown(numpy.ndarray)

    Args:
        arr_drawdown(numpy array): drawdown from function symb_perf_stats or
                                   from ulcer_index.py

    Return:
        UI_MW(numpy.ndarray): Ulcer-Index on moving-window
        applied to arr_drawdown
    """

    import numpy as np

    # apply moving window to arr_drawdown, result is numpy ndarray
    window = 15
    DD_MW = moving_window(arr_drawdown, window)
    DD_MW_square = np.square(DD_MW)
    sum_DD_MW_square = np.sum(DD_MW_square, axis=1)  # sum each row
    UI_MW = np.sqrt(sum_DD_MW_square / window)
    UI_MW = nan_pad(len(arr_drawdown), UI_MW)  # pad with leading NaN

    return UI_MW

In [46]:
def UI_MW_30(arr_drawdown):
    """Calculate Ulcer-Index on moving-window applied to
       arr_drawdown(numpy.ndarray)

    Args:
        arr_drawdown(numpy array): drawdown from function symb_perf_stats or
                                   from ulcer_index.py

    Return:
        UI_MW(numpy.ndarray): Ulcer-Index on moving-window
        applied to arr_drawdown
    """

    import numpy as np

    # apply moving window to arr_drawdown, result is numpy ndarray
    window = 30
    DD_MW = moving_window(arr_drawdown, window)
    DD_MW_square = np.square(DD_MW)
    sum_DD_MW_square = np.sum(DD_MW_square, axis=1)  # sum each row
    UI_MW = np.sqrt(sum_DD_MW_square / window)
    UI_MW = nan_pad(len(arr_drawdown), UI_MW)  # pad with leading NaN

    return UI_MW

In [15]:
def nan_pad(arr_len, target_arr):
    """Pad target_arr with leading numpy.nan to length arr_len.

    Args:
        arr_len(int): length of return array
        target_arr(numpy.ndarray): array to be padded with leading numpy.nan

    Return:
        padded_target_arr(numpy.ndarray): target_arr padded to length arr_len
    """

    import numpy as np

    pad_len = arr_len - len(target_arr)
    pad_arr = np.empty(pad_len, dtype=float)
    pad_arr.fill(np.nan)
    padded_target_arr = np.concatenate((pad_arr, target_arr), axis=0)

    return padded_target_arr

In [48]:
import numpy as np
UI_MW_15 = np.apply_along_axis(UI_MW_15, 0, drawdown)
UI_MW_30 = np.apply_along_axis(UI_MW_30, 0, drawdown)
print(f'UI_MW_15.shape : {UI_MW_15.shape}')
print(f'UI_MW_30.shape : {UI_MW_30.shape}')

UI_MW_15.shape : (252, 1873)
UI_MW_30.shape : (252, 1873)


In [51]:
import pandas as pd
df_UI_MW_15 = pd.DataFrame(data=UI_MW_15, columns=symbols)

In [55]:
df.A

Date
2021-08-30    176.278030
2021-08-31    175.021271
2021-09-01    175.001297
2021-09-02    177.564743
2021-09-03    178.821518
                 ...    
2022-08-23    132.639999
2022-08-24    133.669998
2022-08-25    136.009995
2022-08-26    129.869995
2022-08-29    128.110001
Name: A, Length: 252, dtype: float64

In [56]:
df_UI_MW_15.A.head(20)

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14    0.023711
15    0.026262
16    0.027989
17    0.029458
18    0.031080
19    0.036209
Name: A, dtype: float64

In [45]:
a = x[:,-1]
np.savetxt('C:/Users/ping/Desktop/ZWS_UI_MW.csv', a, delimiter=",")
